<a href="https://colab.research.google.com/github/BhagyashriKasabe/EVcouplings/blob/master/NLP_and_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install faiss-cpu
import pandas as pd
import faiss
import numpy as np
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 88.6 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from google.colab import files
uploaded = files.upload()


Saving sample_docs.csv to sample_docs.csv
Saving support_tickets.csv to support_tickets.csv


In [6]:
import pandas as pd

df = pd.read_csv('sample_docs.csv')
df.head()

,doc_id,section_title,content
0,1,Introduction,This device helps users track their activity l...
1,2,Reset Instructions,"To reset the device, hold the power button for..."
2,3,Battery Issues,"If the battery drains quickly, try reducing sc..."
3,4,Connectivity Problems,Ensure Bluetooth is enabled. Restart both your...
4,5,Warranty,The product includes a one-year limited warran...


In [12]:
def chunk_text(text, chunk_size=200):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = []
for _, row in df.iterrows():
    for ch in chunk_text(row["content"]):
        chunks.append(ch)

print("Total chunks:", len(chunks))

Total chunks: 5


In [14]:
# STEP 4: Create Embeddings & Vector Database
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = np.array([model.encode(ch) for ch in tqdm(chunks)])

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 5/5 [00:00<00:00, 11.90it/s]


In [18]:
# STEP 5: Retrieval Function (RAG)
def retrieve_docs(query, k=3):
    q_emb = model.encode(query)
    D, I = index.search(np.array([q_emb]), k)
    return [chunks[i] for i in I[0]]
retrieve_docs("How do I reset my device?")


['To reset the device, hold the power button for 10 seconds until the light blinks...',
 'Ensure Bluetooth is enabled. Restart both your phone and the device...',
 'If the battery drains quickly, try reducing screen brightness and disabling GPS...']

In [19]:
# STEP 6: LLM Answer Generator (using HuggingFace)
from transformers import pipeline
qa = pipeline("text-generation", model="gpt2")


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [20]:
def answer_query(query):
    context = " ".join(retrieve_docs(query))
    prompt = f"Use the documentation to answer:\nContext: {context}\nQuestion: {query}\nAnswer:"
    return qa(prompt, max_length=200)[0]['generated_text']


In [21]:
# STEP 7: Auto-Generate FAQs
prompt = "Generate 5 FAQs based on this content:\n" + " ".join(chunks)
faqs = qa(prompt, max_length=300)
print(faqs)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Generate 5 FAQs based on this content:\nThis device helps users track their activity levels and monitor health metrics... To reset the device, hold the power button for 10 seconds until the light blinks... If the battery drains quickly, try reducing screen brightness and disabling GPS... Ensure Bluetooth is enabled. Restart both your phone and the device... The product includes a one-year limited warranty covering manufacturing defects...\nThis product was discontinued on September 15, 2018\nThis product was discontinued on September 15, 2018 This product was discontinued on September 15, 2018\nThis product was discontinued on September 15, 2018 This product is discontinued on September 15, 2018 This product was discontinued on September 15, 2018 This product was discontinued on September 15, 2018\nThis product was discontinued on September 15, 2018 This product was discontinued on September 15, 2018 This product was discontinued on September 15, 2018 This product 

In [22]:
# STEP 8: Support Ticket Classification (NLP)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = tickets["description"]
y = tickets["issue"]

tfidf = TfidfVectorizer()
X_vec = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2)

clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Accuracy:", clf.score(X_test, y_test))


NameError: name 'tickets' is not defined

In [24]:
# STEP 8: Support Ticket Classification (NLP)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

tickets = pd.read_csv('support_tickets.csv')

X = tickets["description"]
y = tickets["issue"]

tfidf = TfidfVectorizer()
X_vec = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2)

clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.0


In [25]:
# STEP 9: Build a Support Query → Answer System
def support_bot(user_query):
    # classify
    category = clf.predict(tfidf.transform([user_query]))[0]

    # retrieve relevant docs
    docs = retrieve_docs(user_query)

    # generate answer
    answer = answer_query(user_query)

    return category, docs, answer
